In [6]:
import json
# Load the dictionary containing filenames and titles from the JSON file
with open('data_final_stopword_lemma.json', 'r') as json_file:
    data = json.load(json_file)
# Extract titles from the dictionary

In [25]:
# Initialize a list to store verbs
verbs_list = []

# Specify the file path of the text file
file_path = "most_common_verbs.txt"

# Read the text file and save each line (verb) into the list
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        verb = line.strip()  # Remove leading/trailing whitespace
        verbs_list.append(verb)

# Print the list of verbs
print("List of verbs:")
most_common_verbs = verbs_list[:14]
print(most_common_verbs)

List of verbs:
['voir', 'trouver', 'donner', 'venir', 'pouvoir', 'prendre', 'devoir', 'vouloir', 'mettre', 'falloir', 'savoir', 'note', 'ordre', 'aller']


In [26]:
#load model top2vec
# decider d'eliminer certains categories de documents qui ne donnent que des contenus administratif et d'organisation
# et enlever aussi des mots bruyants qui apparait dans le resultat final de top2vec mais ne donnent aucun sens ou sujet
import nltk
nltk.download('punkt')
import spacy
from nltk.corpus import stopwords
from collections import defaultdict
count = 0

filter_doc = []#"actes-de-la-soci", "liste-des-membres", "liste-générale-des-membres", "procès-verbaux", "procès-verbal"] 
#               "rapport-financier", "rapport-du-trésorier" ]

corpus_man_filtered=[]
len_dict = {}

for article in data:
    all_sub_text_not_found = True
    text = article["FolderName"]
    doc = article["TextData"]
    for elim in filter_doc:
        if elim in text:
            if elim in len_dict:
                len_dict[elim] += article["Words"]/63453
            else:
                len_dict[elim] = article["Words"]/63453
            all_sub_text_not_found = False
    if all_sub_text_not_found:
        corpus_man_filtered.append(doc)
        
# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Get the French stopwords
french_stopwords_spacy = nlp.Defaults.stop_words
# Download the French stopwords dataset
nltk.download("stopwords")
# Load the nltk stopwords
fr_stopwords = set(stopwords.words("french"))
en_stopwords = set(stopwords.words("english"))

file_path = 'noise_word'  # Replace with the path to your file
noise_word_list = []

with open(file_path, 'r') as file:
    for line in file:
        # Remove leading and trailing whitespace and add the word to the list
        noise_word_list.append(line.strip())

for n in noise_word_list:
    fr_stopwords.add(n)
for n in en_stopwords:
    fr_stopwords.add(n)
for v in most_common_verbs:
    fr_stopwords.add(v)
    
i=0
preprocessed_corpus=[]
for text in corpus_man_filtered:
    words = text.split()
    filtered_words = [word.lower() for word in words 
                      if word.lower() not in fr_stopwords 
                      and word.lower() not in french_stopwords_spacy]
    preprocessed_text = " ".join(filtered_words)
    preprocessed_corpus.append(preprocessed_text)
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
'faire' in french_stopwords_spacy

False

In [27]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Sample corpus (list of documents)
corpus = preprocessed_corpus

# Tokenize and preprocess the documents
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]
filtered_corpus = [[word for word in doc if word not in stop_words] for doc in tokenized_corpus]

# Create a dictionary and a corpus from the preprocessed text
dictionary = corpora.Dictionary(filtered_corpus)
corpus_bow = [dictionary.doc2bow(doc) for doc in filtered_corpus]

# Build the LDA model
lda_model = LdaModel(corpus_bow, num_topics=5, id2word=dictionary, passes=15)

# Print the topics and their associated words
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic_words}")

# Compute the coherence score to evaluate the model
coherence_model_lda = CoherenceModel(model=lda_model, texts=filtered_corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_lda}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0: 0.006*"faire" + 0.004*"grand" + 0.003*"bien" + 0.003*"roi" + 0.002*"pays" + 0.002*"jour" + 0.002*"travail" + 0.002*"français" + 0.002*"cochinchine" + 0.002*"chinois"
Topic 1: 0.003*"vietnamien" + 0.003*"nam" + 0.003*"chinois" + 0.003*"nguy" + 0.003*"langue" + 0.002*"étude" + 0.002*"thi" + 0.002*"bien" + 0.002*"histoire" + 0.002*"grand"
Topic 2: 0.005*"grand" + 0.005*"faire" + 0.004*"bien" + 0.003*"homme" + 0.003*"jour" + 0.003*"nom" + 0.002*"petit" + 0.002*"année" + 0.002*"chinois" + 0.002*"long"
Topic 3: 0.014*"société" + 0.007*"membre" + 0.007*"comité" + 0.006*"président" + 0.005*"séance" + 0.004*"général" + 0.004*"saigon" + 0.003*"bulletin" + 0.003*"etude" + 0.003*"secrétaire"
Topic 4: 0.009*"saigon" + 0.004*"faire" + 0.004*"annamite" + 0.004*"cochinchine" + 0.004*"amiral" + 0.003*"lettre" + 0.003*"france" + 0.003*"pari" + 0.003*"bien" + 0.002*"français"
Coherence Score: 0.4201545567750662


In [28]:
# Print the topics and their associated keywords (without weights)
for topic_id, topic_words in lda_model.print_topics(num_words=10):  # Adjust num_words as needed
    keywords = ",".join(word for word, weight in lda_model.show_topic(topic_id))
    print(f"Topic {topic_id}: {keywords}")

Topic 0: faire,grand,bien,roi,pays,jour,travail,français,cochinchine,chinois
Topic 1: vietnamien,nam,chinois,nguy,langue,étude,thi,bien,histoire,grand
Topic 2: grand,faire,bien,homme,jour,nom,petit,année,chinois,long
Topic 3: société,membre,comité,président,séance,général,saigon,bulletin,etude,secrétaire
Topic 4: saigon,faire,annamite,cochinchine,amiral,lettre,france,pari,bien,français


In [30]:

from gensim import similarities 
# Create a similarity index for the documents
index = similarities.MatrixSimilarity(lda_model[corpus_bow])  # Create a similarity index

# Print the most related documents of each topic and their length
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    for doc_id, similarity in enumerate(index[lda_model[corpus_bow[topic_id]]]):
        if similarity > 0.8:  # Only consider documents with non-zero similarity
            doc_length = len(corpus[doc_id].split())  # Get the length of the document
            print(f"  Document {doc_id} (Length: {doc_length} words) - Similarity: {similarity:.2f} - {data[doc_id]['Title']}")


Topic 0:
  Document 0 (Length: 6262 words) - Similarity: 1.00 - vo nguy amiral gialong
  Document 1 (Length: 6109 words) - Similarity: 1.00 - vocabulaire annamite français
  Document 2 (Length: 1799 words) - Similarity: 1.00 - vue ensemble sculpture modelage chine durant vingt siècles
  Document 3 (Length: 11273 words) - Similarity: 1.00 - vérification dates inscriptions monuments khmers seconde partie
  Document 4 (Length: 2875 words) - Similarity: 1.00 - etude procédés fonderie artisanale viet nam
  Document 5 (Length: 1501 words) - Similarity: 0.94 - etude monuments représentatifs art français saigon
  Document 23 (Length: 485 words) - Similarity: 1.00 - édicule birman laos
  Document 40 (Length: 3131 words) - Similarity: 1.00 - autour petit pied chinoise
  Document 42 (Length: 55 words) - Similarity: 1.00 - ruines angkor
  Document 54 (Length: 74 words) - Similarity: 0.94 - bibliographie
  Document 58 (Length: 1128 words) - Similarity: 1.00 - biographie chau van tiêp général gia lo

  Document 192 (Length: 66806 words) - Similarity: 1.00 - trois royaumes traduction notes commentaires nghiêm to louis ricaud
  Document 193 (Length: 769 words) - Similarity: 1.00 - vietnamiens insectes
  Document 194 (Length: 4374 words) - Similarity: 1.00 - villages plateau bolovens
  Document 217 (Length: 307 words) - Similarity: 1.00 - miniature chinoise noyaux sculptés
  Document 219 (Length: 20954 words) - Similarity: 1.00 - nguy nh khiêm porte parole sagesse populaire ch am ng thi recueil poèmes langue nationale retraite nuage blanc
  Document 220 (Length: 766 words) - Similarity: 1.00 - note additionnelle étude nguy nh lai musique sino vietnamienne chants populaires viet nam
  Document 222 (Length: 2901 words) - Similarity: 1.00 - soulèvement sœurs tr ng travers textes folklore vietnamien
  Document 226 (Length: 272 words) - Similarity: 0.95 - note soie fine peaux buffles
  Document 228 (Length: 260 words) - Similarity: 1.00 - note fer nord cambodge
  Document 231 (Length: 360 

In [48]:

# Build the LDA model
lda12 = LdaModel(corpus_bow, num_topics=12, id2word=dictionary, passes=15)
lda_model = lda12
# Print the topics and their associated words
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic_words}")

# Compute the coherence score to evaluate the model
coherence_model_lda = CoherenceModel(model=lda_model, texts=filtered_corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_lda}")

Topic 0: 0.007*"cachet" + 0.006*"caractère" + 0.004*"forme" + 0.003*"sceau" + 0.003*"grand" + 0.003*"che" + 0.003*"ang" + 0.002*"ancien" + 0.002*"han" + 0.002*"chinois"
Topic 1: 0.007*"mer" + 0.006*"sud" + 0.005*"pays" + 0.004*"che" + 0.004*"tchen" + 0.004*"ang" + 0.003*"nord" + 0.003*"nom" + 0.003*"nan" + 0.003*"tan"
Topic 2: 0.015*"société" + 0.011*"saigon" + 0.008*"comité" + 0.008*"membre" + 0.007*"séance" + 0.007*"président" + 0.006*"général" + 0.004*"etude" + 0.004*"secrétaire" + 0.004*"bulletin"
Topic 3: 0.005*"jarre" + 0.004*"vietnam" + 0.004*"saigon" + 0.003*"perle" + 0.003*"van" + 0.003*"lao" + 0.003*"objet" + 0.003*"poterie" + 0.003*"forme" + 0.002*"tesson"
Topic 4: 0.003*"village" + 0.001*"nam" + 0.001*"viet" + 0.001*"system" + 0.001*"central" + 0.001*"vietnamese" + 0.001*"area" + 0.001*"social" + 0.001*"vith" + 0.001*"northern"
Topic 5: 0.009*"cau" + 0.006*"ang" + 0.005*"roi" + 0.005*"faire" + 0.004*"samtec" + 0.004*"troupe" + 0.004*"pays" + 0.003*"grand" + 0.003*"homme" + 

In [35]:
lda12 = lda_model

In [49]:
# Print the topics and their associated keywords (without weights)
for topic_id, topic_words in lda_model.print_topics(num_words=10):  # Adjust num_words as needed
    keywords = ",".join(word for word, weight in lda_model.show_topic(topic_id))
    print(f"Topic {topic_id}: {keywords}")

Topic 0: cachet,caractère,forme,sceau,grand,che,ang,ancien,han,chinois
Topic 1: mer,sud,pays,che,tchen,ang,nord,nom,nan,tan
Topic 2: société,saigon,comité,membre,séance,président,général,etude,secrétaire,bulletin
Topic 3: jarre,vietnam,saigon,perle,van,lao,objet,poterie,forme,tesson
Topic 4: village,nam,viet,system,central,vietnamese,area,social,vith,northern
Topic 5: cau,ang,roi,faire,samtec,troupe,pays,grand,homme,royal
Topic 6: faire,grand,bien,jour,homme,roi,pays,chinois,annamite,lettre
Topic 7: thi,nguy,cho,nam,phong,chi,femme,trong,vietnamien,bien
Topic 8: nguy,grand,année,empereur,mandarin,lê,nam,chinois,tri,faire
Topic 9: eau,petit,grand,art,forme,village,bien,terre,maison,pierre
Topic 10: chinois,paddy,histoire,chine,van,auteur,étude,réf,vietnamien,cochinchine
Topic 11: compte,nam,pari,sud,bien,vietnamien,grand,nom,face,licent


In [39]:

from gensim import similarities 
# Create a similarity index for the documents
index = similarities.MatrixSimilarity(lda_model[corpus_bow])  # Create a similarity index

# Print the most related documents of each topic and their length
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    for doc_id, similarity in enumerate(index[lda_model[corpus_bow[topic_id]]]):
        if similarity > 0.95:  # Only consider documents with non-zero similarity
            doc_length = len(corpus[doc_id].split())  # Get the length of the document
            print(f"  Document {doc_id} ({doc_length} mots) - Similarité : {similarity:.2f} - {data[doc_id]['FolderName']}")


Topic 0:
  Document 0 (6262 mots) - Similarité : 1.00 - vo-di-nguy-1745-1801-amiral-de-gialong_1943_tome_xviii_images
  Document 58 (1128 mots) - Similarité : 0.98 - biographie-de-chau-van-tiêp-général-de-gia-long_1910_semestre_2_images
  Document 91 (5366 mots) - Similarité : 0.97 - de-bordeaux-à-hué-au-temps-de-s-m-gia-long-publication-de-la-relation-du-second-voyage-du-henri-à-la-cochinchine_1932_tome_vii_images
  Document 157 (5202 mots) - Similarité : 0.96 - la-région-de-camau-vers-1898_1968_tome_xliii_images
  Document 183 (3159 mots) - Similarité : 0.98 - les-premières-relations-entre-le-viêt-nam-et-les-etats-unis-d-amérique_1962_tome_xxxvii_images
  Document 272 (292 mots) - Similarité : 0.99 - mémoire-laissé-à-la-postérité-comme-expression-d-adieux-et-rédigé-pendant-sa-dernière-maladie-par-mai-thanh-cong-chef-de-2e-classe-du-canton-de-bao-thanh-province-de-bentré_1911_semestre_1_images
  Document 301 (190 mots) - Similarité : 0.97 - sonnets-1-le-bol-de-riz-2-le-temple-de-madur

In [43]:

# Build the LDA model
lda24 = LdaModel(corpus_bow, num_topics=24, id2word=dictionary, passes=15)
lda_model = lda24
# Print the topics and their associated words
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic_words}")

# Compute the coherence score to evaluate the model
coherence_model_lda = CoherenceModel(model=lda_model, texts=filtered_corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_lda}")

Topic 10: 0.003*"poste" + 0.003*"paillote" + 0.002*"xay" + 0.002*"délégation" + 0.002*"village" + 0.001*"phu" + 0.001*"xine" + 0.001*"milicien" + 0.001*"bou" + 0.001*"méra"
Topic 1: 0.008*"annamite" + 0.008*"lettre" + 0.007*"amiral" + 0.007*"faire" + 0.005*"saigon" + 0.004*"hanoi" + 0.004*"commandant" + 0.004*"garnier" + 0.004*"tonkin" + 0.004*"français"
Topic 17: 0.022*"bahnar" + 0.011*"jarai" + 0.010*"vietnamien" + 0.008*"sedang" + 0.008*"jarer" + 0.006*"khmer" + 0.005*"mot" + 0.005*"langue" + 0.005*"voyelle" + 0.004*"consonne"
Topic 8: 0.002*"tibétain" + 0.001*"burnouf" + 0.001*"langue" + 0.001*"lévi" + 0.001*"indianisme" + 0.001*"sanskrit" + 0.001*"tokharien" + 0.001*"sogdien" + 0.001*"indien" + 0.001*"recherche"
Topic 21: 0.003*"arrêté" + 0.002*"nhlong" + 0.002*"nhoà" + 0.002*"sadec" + 0.001*"délégation" + 0.001*"canton" + 0.001*"inspection" + 0.001*"càmau" + 0.001*"contrainte" + 0.001*"luin"
Topic 16: 0.002*"pot" + 0.002*"chaux" + 0.002*"cuy" + 0.001*"chique" + 0.001*"annamite" +

In [50]:
# Print the topics and their associated keywords (without weights)
for topic_id, topic_words in lda24.print_topics(num_words=10):  # Adjust num_words as needed
    keywords = ",".join(word for word, weight in lda24.show_topic(topic_id))
    print(f"Topic {topic_id}: {keywords}")

Topic 11: société,membre,comité,séance,président,saigon,général,faire,bulletin,travail
Topic 13: nam,carte,saigon,vietnamien,nom,société,viet,sud,duong,van
Topic 3: association,pinus,pin,classe,social,dalat,chef,structure,réclusion,individu
Topic 4: paddy,vietnam,kuê,van,lao,saigon,demande,carte,bull,cambodge
Topic 15: faire,grand,roi,jour,thi,année,bien,village,mandarin,mois
Topic 7: faire,grand,bien,siam,fort,eau,roi,pays,français,lettre
Topic 23: nguy,nam,thi,grand,lê,vietnamien,chinois,tri,année,texte
Topic 16: pot,chaux,cuy,chique,annamite,chine,poisson,vai,espèce,anse
Topic 14: société,pari,tonkin,saigon,géographie,cochinchine,chine,carte,indo,français
Topic 9: fromage,chao,soia,géologique,haricot,azote,mine,sauce,graine,fermenter
Topic 18: jarre,forme,petit,long,objet,poterie,partie,région,terre,perle
Topic 17: bahnar,jarai,vietnamien,sedang,jarer,khmer,mot,langue,voyelle,consonne
Topic 10: poste,paillote,xay,délégation,village,phu,xine,milicien,bou,méra
Topic 0: rue,face,mélané

In [47]:

from gensim import similarities 
# Create a similarity index for the documents
index = similarities.MatrixSimilarity(lda_model[corpus_bow])  # Create a similarity index

# Print the most related documents of each topic and their length
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    for doc_id, similarity in enumerate(index[lda_model[corpus_bow[topic_id]]]):
        if similarity > 0.99:  # Only consider documents with non-zero similarity
            doc_length = len(corpus[doc_id].split())  # Get the length of the document
            print(f" {doc_id} ({doc_length} mots) - Similarité : {similarity:.2f} - {data[doc_id]['Title']}")


Topic 0:
 0 (6262 mots) - Similarité : 1.00 - vo nguy amiral gialong
Topic 1:
 1 (6109 mots) - Similarité : 1.00 - vocabulaire annamite français
 98 (675 mots) - Similarité : 0.99 - eloge funèbre mgr adran roi gia long document
Topic 2:
 2 (1799 mots) - Similarité : 1.00 - vue ensemble sculpture modelage chine durant vingt siècles
Topic 3:
 3 (11273 mots) - Similarité : 1.00 - vérification dates inscriptions monuments khmers seconde partie
 90 (5280 mots) - Similarité : 1.00 - ng jeu échecs vietnamiens
 173 (1993 mots) - Similarité : 1.00 - juifs chine solution question juive
 240 (950 mots) - Similarité : 1.00 - notes cachet or découvert annam
 299 (40392 mots) - Similarité : 0.99 - sigillographie sino annamite planches photos
 356 (27090 mots) - Similarité : 1.00 - grande java marco polo cochinchine etude géographie historique après sources chinoises arabes
 367 (6479 mots) - Similarité : 1.00 - etude vérification dates inscriptions monuments khmers
 368 (3423 mots) - Similarité : 1.